***CATEGORY THEORY WITH JULIA***
-----

# 1. Types and Categories in Julia

In programming languages, we can define a category by considering types to be objects
and functions to be morphisms.

## 1.1 Types in Julia
In Julia, types can be concrete or abstract. Only concrete type can be instantiated, while abstract
types are ways of grouping these types. For example, `Int` and `Float64` are concrete types, while
`Number` is an abstract type that contains both, i.e. `Int` and `Float64` are subtypes of `Number`.

In [4]:
using Pkg
Pkg.activate(".")

  Activating project at `~/MEGA/EMAP/PhDThesis/notes/FunctionalProgrammingCategoryTheory/julia`


In [5]:
Int <: Number, Float64 <: Number

(true, true)

In [6]:
isconcretetype(Int)

true

In [7]:
isabstracttype(Real)

true

It's easy to create an abstract type in Julia. 

In [8]:
abstract type MyTypes end

Types can also be composite or primitive. A primitive type is a concrete type where it's definition is given in bits.
Primitive types are the ones you would imagine, e.g. `Float16`, `Float32`, `Float64`, `Bool`, `Int8` and so on.

In Julia, one can define his own primitive type, but this is not usually optimal, since it's not native to the LLVM (the compiler structure used in Julia).

Composite types are types composed of other types. In Julia, these are the so called `structs`.

In [9]:
struct MyType <: MyTypes
    a::Int
    b::Real
    c
end

In [10]:
MyType <: MyTypes

true

Another way of constructing types is with operations such as `Union`. 

In [11]:
IntOrNothing = Union{Int, Nothing}

1 isa IntOrNothing

true

Another possible contruction is with `Tuple`. 

In [12]:
Tuple2Int = Tuple{Int,Int}

(1,2) isa Tuple2Int

true

In [13]:
MyType2 = Tuple{MyType,MyType}

(MyType(1,2,3),MyType(3,3,"o")) isa MyType2

true

Another interesting way of creating types with tuples is by using `Vararg`. In this case, we 
can do:

In [14]:
MyTypeTuple = Tuple{MyType, Vararg{MyType}}

(MyType(1,2,3),MyType(1,2,3),MyType(3,3,"o")) isa MyTypeTuple

true

Another very important aspect of types are the so called parametric types. 

In [15]:
struct NewType{T} <: MyTypes
    x::T
    y::T
end

Note that we just created a plethora of new types with the caveat that
they all have the fields `x` and `y`, and both fields have the same type.

In [16]:
NewType(10,10)

NewType{Int64}(10, 10)

You can also specify a supertype for the parameter, for example:

In [17]:
struct NewType2{T<:Real}
    x::T
    y::T
end

function f(x::NewType2{T}) where T <: Int
    x.x+x.y
end

f (generic function with 2 methods)

An important thing to note here is that in parametric types is that subtyping behaves a bit different than
for non-composite types. See:

In [18]:
@show NewType(10,10) isa NewType{Int}
@show NewType(10,10) isa NewType{Real};

NewType(10, 10) isa NewType{Int} = true
NewType(10, 10) isa NewType{Real} = false


Although `Int <: Real`, it's not true that `NewType{Int} <: NewType{Real}`. Yet, we can say
that `NewType{Int} <: NewType`.

In [19]:
NewType{Int} <: NewType

true

Here the use of tuples can be helpful. Tuples allow the use of parameters, and the subtyping works for the elements also. 

In [20]:
Tuple{Int, Int} <: Tuple{Real, Real}

true

Lastly, note the following: 

In [21]:
@show typeof(NewType{Int})
@show typeof(NewType)
@show DataType <: Type;

typeof(NewType{Int}) = DataType
typeof(NewType) = UnionAll
DataType <: Type = true




In Julia, even types have types!
Moreover, what is the difference between a DataType and UnionAll?
You can think that a DataType can infer how much memory will be needed for the type, while
a UnionType can't. Why?

Note that for `NewType{Int}` we know that two ints will have to be stored in memory, hence
we know the amount of bits necessary. Yet, for `NewType`, we cannot predict how much bits will be used,
since the parameter type could be pretty much anything.

But what about `Union{Int,Float64}` and `Tuple{Int,Float64}?

In [22]:
@show typeof(Union{Int,Float64})
@show typeof(Union{Int,Real})
@show typeof(Tuple{Int,Float64});

typeof(Union{Int, Float64}) = Union
typeof(Union{Int, Real}) = DataType
typeof(Tuple{Int, Float64}) = DataType


Note that in the first example, we have a type `Union`, which makes sense, since we are either going to allocate the
size of an Int or a Float. But what about the second union? In the second case, Julia is just turning
`Union{Int,Real}` in the type `Real`, sort of just saying "allocate the larger memmory.

Lastly, the Tuple is also a DataType, since we know how much each element will store.


## 1.2 Value Types

Types in Julia are very useful due to multiple-dispatch. It would be very useful if one could also dispatch on values,
for example, a function `f(x::Bool)` with two methods, one for `x == True` and another for `x == False`.

This can be done using `Val`. This is already implemented in Julia, but here is how one could implement it.

In [23]:
struct MyVal{x}
end
MyVal(x) = MyVal{x}()

@show MyVal(1);

MyVal(1) = MyVal{1}()


In [24]:
f(::Val{true}) = 1
f(::Val{false}) = 0

f(Val(true)) ,f(Val(false)) 

(1, 0)

Note that this is easily abused, and must be used carefully.
The proper way to use it is such that the compiler knows what is going to be passed.

## 1.3 Initial and Terminal Objects

In Category Theory, the initial object is one where there is a unique morphism
leaving it to any other object. In the case o $\mathbf{Set}$, this is the empty set, since
every morphism from the empty set is the "same" uncallable function.

In Julia, the initial object is the type `::Union{}`.
Note that this type is known as `Base.Bottom`, and it's actually a subtype of every type in Julia.

In [1]:
Union{} <: Int, Union{} <: Float64, Union{} <: Nothing

(true, true, true)

Since `Union{}` is a subtype of every other type, it behaves in the same way as the empty set,
which is a subset of every other set.

In [4]:
absurd(x::Union{}) = "anything";

*Ex falso sequitur quodlibet*. 

The exact opposite of `Union{}` is the `Any` type, i.e. any value is of type `Any`. 

Now, in the other end of the spectrum, we have a terminal object, which is an object that
for every object in the category, there is only one morphism arriving to it.

In $\mathbf{Set}$, this terminal object is the singleton set, i.e., the set with a single object.
Now, you may wonder, "but we have an infinite number of singletons, which one is the terminal?". The answer is
that they all are "the same" up to an isomorphism. 
Hence, the terminal object is the unique up to an isomorphism.

But what about in Julia? The terminal object would be a type that has only a single possible "value", also
called a singleton type. Hence,
`Int` or `Float64` or `Char` are not possibilities, since they have many elements.
We could construct such a type. See:


In [27]:
struct Terminal end

Terminal()

Terminal()

In [28]:
Base.issingletontype(Terminal)

true

In Julia, the `Nothing` type is already a type with a singleton, which is the value `nothing`. 
So is the `Tuple{}` type, which only has `()` as element.

In [29]:
Base.issingletontype(Nothing),Base.issingletontype(Tuple{})

(true, true)

## 1.4 Product and Coproduct of Types

As we already talked about, types are objects in programming languages such as Julia, and we can
create product objects and coproduct objects, i.e. product types and coproduct types.

So what are they? We actually already talked about them. The `Tuple{Type1, Type2}` is the product
of `Type1` and `Type2`, while `Union{Type1,Type2}` is the coproudct.

In [17]:
Tuple{Int,Int}

Tuple{Int64, Int64}

## 1.5 Type Classes and Traits in Julia

As we said, Julia can defined types in a hierarchical nature, but it does not provide a
"native" way of grouping distincting types in classes.
Remember, we could say that some types belong to the same class if they share some **traits**.

The benefit of having a class of types is that once we know that a certain type pertains
to a certain class, we can then use what we know about such class.
For example, we could think of a class `Eq` which would consist of all types in Julia
where the `==` operator is defined.


In [11]:
struct Point2
    x::Real
    y::Real
end

Base.isequal(p::Point2, q::Point2)::Bool = p.x == q.x && p.y == q.y ? true : false

p = Point2(1,1)
q = Point2(1,1)
q_ = Point2(2,1)

p == q, p == q_

(true, false)

In the implementation above, we've defined a new type `Point2`, and such type would now belong to the class `Eq`.
The question is, how can we define such class in Julia?

What we need is a way to group types that go beyond the simple type hierarchy. To do this,
we use a design pattern known (in Julia) as Holy Trait (note that "Holy" is the surname of Tim Holy, the creator of this idea).

In [4]:
abstract type Eq end
struct IsEq <: Eq end
struct IsNotEq <: Eq end

In [5]:
Eq(::Type) = IsNotEq()
Eq(::Type{<:Point2}) = IsEq()
Eq(::Type{<:String}) = IsEq()
Eq(::Type{<:Number}) = IsEq();

In the code above, we've implemented the class `Eq`.
First, we used `Eq(::Type)= IsNotEq()`, which means that, by default, a type does NOT belong to the `Eq` class.
Next, we used `Eq(::Type{<:Point2})= IsEq()` and others to specify that such types indeed were from class `Eq`.

At first this might not look very useful, yet, it can be, specially if you think of multiple-dispatch.

Consider the following example.

In [6]:
abstract type Geometric end

struct Circle <:Geometric
    r::Real
end
struct Rect <:Geometric
    l::Real
    h::Real
end

struct Trapz <:Geometric
    l::Real
    h::Real
    θ::Real
end
struct Square <:Geometric
    l::Real
    h::Real
    Square(l) = new(l,l)
end

Now, we want to implement a function such as `area` that is
computed by multiplying `l*h`. Note that all shapes have this property, but `Circle`.
Yet, all of them share the `Geomtric` type, hence, we cannot just dispatch on it.

How to solve this? We could perhpas create an `if x is Circle`, but this is not very good,
cause we could add a new geometric shape like `Ellipse` which would also not have the `l` and `h`...



In [7]:
abstract type HasHeightLengthTrait end
struct HasHeightLength <: HasHeightLengthTrait end
struct NotHasHeightLength <: HasHeightLengthTrait end

In [8]:
HasHeightLengthTrait(::Type) = NotHasHeightLength()
HasHeightLengthTrait(::Type{<:Rect})   = HasHeightLength()
HasHeightLengthTrait(::Type{<:Trapz})  = HasHeightLength()
HasHeightLengthTrait(::Type{<:Square}) = HasHeightLength()

HasHeightLengthTrait

In [9]:
hasheightlength(x::T) where T<: Geometric  = hasheightlength(HasHeightLengthTrait(T), x)
hasheightlength(::HasHeightLength, x) = true
hasheightlength(::NotHasHeightLength, x) = false

hasheightlength (generic function with 3 methods)

In [10]:
area(x::T) where T<: Geometric  = area(HasHeightLengthTrait(T), x)
area(::HasHeightLength, x) = x.l * x.h
area(::NotHasHeightLength, x) = error("Area function not implemented.")

s = Square(1)
area(s)

1

## 1.6 Enumerations

The @enum macro create an enumeartion struct. This is nothing more than a type suit where
the possible values are enumerated. Look the example below.

In [1]:
@enum suit clubs diamonds hearts spades
suit

Enum suit:
clubs = 0
diamonds = 1
hearts = 2
spades = 3

In [2]:
instances(suit)
suit(1), Int(diamonds)

(diamonds, 1)

Note that, for example, `diamonds` is of type `suit`, and it's not a type itself, but an instance of `suit`. Similar
to how  `1` is an instance of `Int`.

## 1.7 What it means to be a value of type T?

As pointed out, in programming we have a category $\textbf{Types}$ where types are objects and functions
are morphisms.
If `Int` is an object, then what is a value of `Int` represented in Category Theory?

One might be tempted to say "well, it's just an element of the object". The problem is that, in Category Theory,
we are not "allowed" to peek inside objects. We can only talk about morphisms, objects, and things of higher abstraction (e.g. functors).

To solve this, note that we can think of a type `T` as a set, and we can show that
there is an isomorphism between the values of `T` and the set of morphisms (functions) from
the terminal type `Terminal` to type `T`. Why? Remember, a terminal type `Terminal` is
a type containing only a singleton value. In the example of `Terminal`, such singleton is `Terminal()`.

Hence, for each `a::A`, we can define a function `fa(x::Terminal) = a`. Since the only value
of `Terminal` is `Terminal()`, this means that each function from `Terminal` to `A` is exactly
a function that takes `Terminal()` to a value of `A`. Therefore, the set of morphisms from
`Terminal` to `A` is isomorphic to the set of values of `A`.

Now, consider the following code:

In [24]:
struct T
    x::Int
    y::Int
end
a = (1,2)
t = T(a...)

T(1, 2)

When we created such struct, Julia implicitly created a "value" constructor, i.e. 
a function `T(a::Tuple{Int,Int})` that returns values of type `T`.
Since a value `a::Tuple{Int,Int}` is just a function `a(x::Terminal)::Tuple{Int,Int}`,
then `T(a)` is actually a function composition `T ∘ a` in the category $\mathbf{Types}$.

Thus, we are able to talk about `T(1,2)` as a morphism (function), without requiring to talk
about elements of sets.

But what about the fields `x` and `y`? For a value of type `t = T(a)`, `t.x` can be understood
as a function `x(t::T)::Int`, and the same for `y`. Note that
`t` is isomorphic to `T ∘ a` , thus
`x(t)` is ismorphic to `x ∘ t = x ∘ T ∘ a`.

Hence, again we've defined `t.x` in terms of a morphism.

Lastly, we can prove that our type `T` is isomorphic to `Tuple{Int,Int}`.
Two objects are isormophic if there is a morphism (function) which is left and right invertible,
meaning, for  `f(x::Tuple{Int,Int})::T`, there is `fl(y::T)::Tuple{Int,Int}` and 
`fr(y::T)::Tuple{Int,Int}` such that:

$$
f_l\circ f = id_{\text{Tuple\{Int,Int\}}}
$$

$$
f\circ f_r = id_{\text{T}}
$$

In our example, this function is just `f(x,y) = T(x,y)`. The inverse of `f`
is `fl(T) = (T.x, T.y)`.